In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API 
(for problem finding)

In [2]:
import requests
import json
import os
from dotenv import load_dotenv

# Make sure you have an .env file with the relevant API keys from OpenAI and Mergeflow 
# Infos under following link: https://blog.bitsrc.io/a-gentle-introduction-to-env-files-9ad424cc5ff4

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
username = 'leonard' # replace by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
query = 'home appliances' # topic of the search
dataset = '&sp=2655' # access dataset from mergeflow, in this case scientific publications
rows = str(5) # first 5 docs only

url = f'https://mergeflow.net/api/v1/{username}/search/getContent?q={query}{dataset}&rows={rows}'

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    #pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 5 documents.


# Problem Finding

In [3]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""You are the Chief Customer Officer of a company in the field of {query}. Your work is highly crucial and you are responsible for indentifiying the most significant problems of your customer."""
        
# Initialize an empty list to store problem messages
problems_list = []

for doc in mergeflow_docs:

    user_prompt1 = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
        I'm interested in exploring problems that will occur in the area of {query} using the concept of the adjacent possible, as outlined by Stuart Kauffman and popularized by Steven Johnson. This approach emphasizes incremental innovation by considering what's just beyond the current technological, cultural, or conceptual boundaries, yet still achievable with existing resources or slight advancements. 
        
        I'm looking for ideas that are not far-fetched but are innovative problems that could occur as a result of what currently exists, tapping into the untapped potential lying just outside the present scope of {query}.

        Given this framework, please analyze the current trends, technologies, consumer behaviors, and market needs in {query}. Identify where there might be gaps or unmet needs that align with the concept of the adjacent possible. Consider the following:

        Technological Advancements: What recent technological developments have occurred, and how might they open new opportunities for innovation or improvement?
        Consumer Behavior Shifts: How have consumer preferences and behaviors evolved recently, and what does this imply about potential future needs or desires?
        Regulatory Changes: Are there any recent or upcoming changes in regulations that might create new opportunities or necessitate innovations?
        Cross-industry Inspiration: Look at how other industries are innovating. Are there any ideas or technologies that could be adapted or extended to fit the context of {query}?

       For the problem you found, please only output the following: 
        - "Title": Title of the problem in max 5 words. 
        - "Explanation": The problem (please write 250-300 words)
        - "User group": Which user group is impacted by this problem the most (please write 20-50 words)
        - "Pain point": What is the specific pain point for this user group resulting from the problem (please write 80-100 words)
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt1}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Append the completion message content to the problems_list
    problems_list.append(completion.choices[0].message.content)
    
    print("Problem:\n")
    # This prints the content of the output in a more readable way
    print(completion.choices[0].message.content)
    print("\n---\n")  # This adds a separator between the content blocks for readability

Problem:

Title: Sustainable Appliance Lifecycle Management

Explanation: As the world shifts towards sustainability and renewable energy sources, the home appliance industry faces the challenge of aligning with these environmental goals. The transition towards more sustainable practices has been accelerated by technological advancements, consumer behavior shifts towards eco-friendliness, and regulatory changes demanding higher energy efficiency and lower emissions. However, a significant gap exists in the lifecycle management of appliances, from production and usage to disposal and recycling. Most home appliances are energy-intensive to produce, have a finite lifespan, and are difficult to recycle, contributing to environmental degradation. Innovations in sustainable materials, energy-efficient designs, and recycling technologies present an adjacent possible for the industry. By developing appliances that are designed for disassembly, using recyclable or biodegradable materials, and i

In [14]:
# second agent: 

# Now, iterate over problems_list to rate each problem
rated_problems_list = []
for problem in problems_list:
    user_prompt2 = f"""
    Before you give me any output, please rate the problems according to the following criteria.
        - Size of user group (how many of the mentioned customers exist) (1 = small target group, 2 = medium target group, 3 = large target group)
        - Relevancy of pain point to the specific industry of {query} and the respective user group (e.g., new burger cooking technology not relevant for factory automation) (1 = not relevant to users, 2 = medium relevant to users, 3 = very relevant to users)
        - How specific is the problem? (1 = very generic problem, 2 = medium generic, 3 = specific and not generic problem)
        
    Be very stringent with your ratings:
        1: Does not meet expectations
        2: Meets expectations adequately
        3: Exceeds expectations significantly
    
    A rating of 3 should be reserved for exceptional performance in the given criterion.

    Given the problem:
    {problem}

    Please repeat the name of the problem and then output the ratings and a one-sentence justification (max 50 words).
    Add up the three ratings you gave to an overall score out of 9.
    
    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt2}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    
    # Assume we store the rated problem details for later use
    rated_problems_list.append(completion.choices[0].message.content)

    # Print the rated problem for immediate feedback
    print("Problem rating:\n")
    print(completion.choices[0].message.content)
    print("\n---\n")


Problem rating:

Title: Sustainable Appliance Lifecycle Management

- Size of user group: 3. The demand for sustainable products is rapidly increasing, making this a large target group.
- Relevancy of pain point: 3. The issue is highly relevant to the home appliance industry and the environmentally conscious consumer group.
- How specific is the problem?: 2. The problem is fairly specific to the appliance industry, although it addresses broad sustainability concerns.

Overall score: 8/9.

Justification: The problem of sustainable appliance lifecycle management directly aligns with the growing consumer demand for eco-friendly products and the industry's need for sustainable practices, though it encompasses a broad range of sustainability issues.

---

Problem rating:

**Problem**: Smart Appliance Connectivity Hurdles

- **Size of user group**: 2  
- **Relevancy of pain point**: 3  
- **How specific is the problem?**: 3  

**Justification**: The user group, while growing, isn't yet as la

In [5]:
problems_and_ratings = [a + b for a, b in zip(problems_list, rated_problems_list)]

def print_menu(options):
    for idx, option in enumerate(options, start=1):
        print(f"{idx}. {option}")
        print("\n---\n")
        
def get_user_choice(num_options):
    while True:
        try:
            choice = int(input("Enter your choice (number): "))
            if 1 <= choice <= num_options:
                return choice
            else:
                print(f"Please enter a number between 1 and {num_options}.")
        except ValueError:
            print("Please enter a valid integer.")

def main():
    options = problems_and_ratings
    print_menu(options)
    choice = get_user_choice(len(options))
    print(f"You chose: {options[choice - 1]}")
    return choice

if __name__ == "__main__":
    chosen_index = main() -1
    chosen_problem = problems_list[chosen_index]
    print("Your choice is:\n" + chosen_problem)

1. Title: Sustainable Appliance Lifecycle Management

Explanation: As the world shifts towards sustainability and renewable energy sources, the home appliance industry faces the challenge of aligning with these environmental goals. The transition towards more sustainable practices has been accelerated by technological advancements, consumer behavior shifts towards eco-friendliness, and regulatory changes demanding higher energy efficiency and lower emissions. However, a significant gap exists in the lifecycle management of appliances, from production and usage to disposal and recycling. Most home appliances are energy-intensive to produce, have a finite lifespan, and are difficult to recycle, contributing to environmental degradation. Innovations in sustainable materials, energy-efficient designs, and recycling technologies present an adjacent possible for the industry. By developing appliances that are designed for disassembly, using recyclable or biodegradable materials, and incorpor

Enter your choice (number): 1
You chose: Title: Sustainable Appliance Lifecycle Management

Explanation: As the world shifts towards sustainability and renewable energy sources, the home appliance industry faces the challenge of aligning with these environmental goals. The transition towards more sustainable practices has been accelerated by technological advancements, consumer behavior shifts towards eco-friendliness, and regulatory changes demanding higher energy efficiency and lower emissions. However, a significant gap exists in the lifecycle management of appliances, from production and usage to disposal and recycling. Most home appliances are energy-intensive to produce, have a finite lifespan, and are difficult to recycle, contributing to environmental degradation. Innovations in sustainable materials, energy-efficient designs, and recycling technologies present an adjacent possible for the industry. By developing appliances that are designed for disassembly, using recyclable or

# Getting data from Mergeflow's API 
(for solution finding)

In [6]:
import requests
import json
import os
from dotenv import load_dotenv # stores API keys for OpenAI and Mergeflow 

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY') # Assign Mergeflow API

# the API call
dataset = '&q=*%5bfundingcompany%7cfundingproject%7cfundingorganization%5d&sp=2572' # access dataset from mergeflow - in this case venture capital investments
rows = str(20) # first 20 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/leonard/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content']
        }
        
        mergeflow_docs.append(current_doc)
        #print(content)        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 20 documents.


In [7]:
#Summarize the content of the 19 documents to reduce text input in next task

import openai
import os
import requests
import json

openai.api_key = os.getenv('OPENAI_API_KEY')

summary_list = []

system_prompt_summary = f"""
You are a highly qualified academic scholar who has much experience in communicating your results to a broad audience. You are able to extract the most important aspects of texts, compress them and rephrase them in a comprehensible way.
"""

for doc in mergeflow_docs:
    user_prompt_summary = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {doc['content']}
    ```
    Please summarize the finding of the text in at most 500 words, and clearly cite the source at the end.
    """
    completion_summary = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt_summary},
            {"role": "user", "content": user_prompt_summary}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    summary_list.append(completion_summary.choices[0].message.content)
    print(completion_summary.choices[0].message.content)
    print("\n---\n")

Land, an innovative electric mobility and portable battery manufacturer based in Cleveland, Ohio, has successfully secured $3 million in funding. The company, which was established in 2020 by Scott Colosimo, is pioneering in the field of energy continuity and mobility. The essence of Land's mission revolves around the development of a transitional e-Moto ecosystem supported by a versatile portable power platform known as CORE. This groundbreaking technology enables the distribution of energy from the battery, facilitating a portable power source that is both on and off the grid.

A significant feature of Land's products is the 'Power Tap' accessory, which, when connected to the electric vehicle, allows users to charge various devices. This includes mobile phones, laptops, and even household appliances, through both USB-C ports and standard plugs. This innovation not only highlights Land's commitment to enhancing mobility and energy accessibility but also underscores the practical utili

Cyclic Materials, a pioneering metals recycling firm based in Kingston, Ontario, Canada, has successfully raised $27 million in Series A funding, elevating its total funds to over $30 million. This significant financial milestone was achieved through the leadership of Energy Impact Partners and BMW i Ventures, alongside contributions from Fifth Wall, Bioindustrial Innovation Canada (BIC), and Planetary Technologies, an existing investor.

Under the guidance of CEO Ahmad Ghahreman, Cyclic Materials is dedicated to establishing a sustainable and domestic supply chain for rare earth elements (REEs) and other essential metals. The company's innovative recycling process aims to reintegrate REEs into the manufacturing cycle, thus mitigating the environmental consequences associated with the global shift towards energy efficiency. REEs play a crucial role in the electrification and decarbonization movement, serving as fundamental components in electric vehicle motors, wind turbines, and a var

Shifted Energy, a company based in Honolulu, Hawaii, that specializes in distributed energy resources (DER) technology, has successfully secured $4.3 million in seed funding. The funding round was led by EPIC Ventures and Kapor Capital, with additional contributions from Buoyant Ventures, Startup Capital Ventures x SBI Fund, and Hunt Development. Since its inception in 2016, Shifted Energy, under the leadership of CEO Forest Frizzell and CTO Olin Lagon, has been at the forefront of developing innovative energy solutions.

The company's primary focus is to facilitate the transition towards more sustainable and efficient energy systems by providing low-cost DER control and monitoring systems. These systems are enhanced with machine-learning data analytics and dispatch optimization software. Shifted Energy's groundbreaking approach involves the creation of virtual power plants by integrating electrical appliances, such as water heaters and electric vehicle (EV) chargers, into a cloud-base

Elephant Energy, a pioneering climate tech company based in Boulder, Colorado, has successfully secured $3.5 million in seed funding to drive its mission forward. The company, founded in 2021 by DR Richardson and Josh Lake, aims to revolutionize home electrification by offering a platform that facilitates the transition of homeowners and contractors to modern electric technologies. This innovative approach not only promises to enhance the efficiency of home appliances but also significantly reduce carbon emissions, contributing to the fight against climate change.

The funding round was spearheaded by Building Ventures and saw contributions from notable investors including Daniel Gulati, Workshop, and Reuben Munger. This financial boost is set to support Elephant Energy's ambitious goal to "electrify everything," starting with residential spaces. By leveraging building science expertise alongside proprietary technology, Elephant Energy is uniquely positioned to assist in upgrading vari

Wikifactory, an innovative online platform headquartered in Copenhagen, Denmark, has successfully raised $2.5 million in a pre-Series A funding round. This recent financial injection, contributed by a mix of existing shareholders and new investors including Lars Seier Christensen's investment vehicle, elevates the company's total funding to nearly $8 million. Under the leadership of Nicolai Peitersen, Wikifactory is pioneering a collaborative environment for a diverse community of developers, designers, engineers, and startups worldwide. The platform facilitates the cooperative development, prototyping, and production of hardware solutions aimed at addressing real-world challenges.

The company's mission is centered around the creation of the "Internet of Production." This concept represents a visionary approach to manufacturing, characterized by a distributed, interoperable, and standards-based framework. It seeks to seamlessly integrate product definitions, software services, and Man

Sense, a company based in Cambridge, Massachusetts, specializing in providing real-time insights into home energy usage through its innovative platform, has successfully completed a Series C funding round, raising $105 million. This recent financial achievement elevates the company's total funding to nearly $157 million. The funding round was spearheaded by Blue Earth Capital and saw significant contributions from a diverse group of investors, including TELUS Ventures, MCJ Collective, Schneider Electric, Energy Impact Partners, Prelude Ventures, and iRobot.

The capital raised in this funding round is earmarked for several strategic initiatives aimed at enhancing the company's market presence and operational capabilities. Primarily, Sense plans to use the funds to drive widespread consumer adoption of its platform. This strategy includes forging partnerships within an ecosystem comprising utilities, home construction firms, and smart home innovators. Additionally, Sense is looking to b

In [8]:
Problem = chosen_problem

print(Problem)

Title: Sustainable Appliance Lifecycle Management

Explanation: As the world shifts towards sustainability and renewable energy sources, the home appliance industry faces the challenge of aligning with these environmental goals. The transition towards more sustainable practices has been accelerated by technological advancements, consumer behavior shifts towards eco-friendliness, and regulatory changes demanding higher energy efficiency and lower emissions. However, a significant gap exists in the lifecycle management of appliances, from production and usage to disposal and recycling. Most home appliances are energy-intensive to produce, have a finite lifespan, and are difficult to recycle, contributing to environmental degradation. Innovations in sustainable materials, energy-efficient designs, and recycling technologies present an adjacent possible for the industry. By developing appliances that are designed for disassembly, using recyclable or biodegradable materials, and incorporati

# Solution Finding

In [9]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY') # Assign OpenAI API

#System prompt (for all users)
system_prompt = f"""You are part of a brainstorming session that wants to come up with a solution to a specific problem. You customer handed you the following problem:
       
        {Problem}
        
        Now you try to figure out a solution in a brainstorming session with the Walt Disney Method: 
        It is a tool for brainstorming ideas. It involves using 3 sequential roles, namely the Dreamer, the Realist, and the Critic.
        The Dreamer: you focus on free association and brainstorming of ideas. Anything goes here. It's your chance to let your imagination run wild! Be enthusiastic and approach the task with questions such as “What would be nice?”, “What would be the optimal situation?” or “What would I wish for?” Assume that everything is possible.
        The Realist: Be realistic and decide which of your ideas are actually possible or practical. What would you need to do in the real world in order to make them happen? Take a pragmatic approach and ask which materials, which knowledge, which techniques there are with which the thoughts of the dreamer can be realised.
        The Critic: Examine proposals and its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. It is about identifying opportunities and risks, pointing out limits and gaps, and expressing constructive and always positive criticism. Try to identify possible sources of error.
        
        Your work is highly crucial and you are responsible of coming up with the next best innovation.
        """

#Dreamer prompt (idea: come up with crazy ideas)
dreamer_solutions = {}
i = 0

user_prompt = f"""
    Consider the text below, delimited by ```, in the area of {query}:
    ```
    {summary_list}
    ```    
  
    Imagine you are the Dreamer. 
    
    Take a deep breath and work on this problem step by step. 

    
    Please develop 5 really specific ideas/visions on how this problem can be solved. The solutions should consider the following points:
    - You can let your imagination run wild and you do not care about possible limitations. It's more important the idea is new then it can really work.  
    - They should not be too general. 
    - The solution should not already exist on the market and be novel. 
    - The ideas should not be too similar to each other
    
    The solution should contain the following contents:
    1) Your solution in max 3 words (title)
    2) Your solution summarized in 3 sentences.
    3) Your solution from the customer perspective (customer journey: how does it work for the customer) 
    4) Why the solution is kind of new (not existing on the market)
    5) Please provide some sources of inspiration that you used from the document excerpts.


    
    """
    
completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.8,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

print("Dreamer Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
dreamer_solutions[i] = completion.choices[0].message
i += 1


#Realist prompt (idea: evaluate if it is possible and what would be needed)
realist_solutions = {}

user_prompt1 = f"""

    Imagine you are the Realist. 
    
    Take a deep breath and work on this problem step by step. 

    
    Consider the following solutions: {dreamer_solutions}.
    
    Please give me the 3 clearly speperated pieces of information for each of the solutions (in total 50 words max):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 1 detail": What would you need to do in the real world in order to make the solution happen? Answer which knowledge, which techniques there are with which the solutions can be realised. Explain in 5 sentences. 
    4) "Evaluation criteria 1": Is the solution actually possible? Answer can be 1,2 or 3 (1 = not possible, 2 = maybe possible, 3 = possible) (be the most critical person ever. Only give max. 1 of the 3 solution a 3)

    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt1}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print("Realist Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
realist_solutions[0] = completion.choices[0].message


#Critics prompt (idea: evaluate if there are a lot potential sources of error)
critics_solutions = {}

user_prompt2 = f"""

    Imagine you are the Critic. 
    
    Consider the following solutions: {realist_solutions}.
    
    Take a deep breath and work on this problem step by step. 
    
    Please give me the following 4 clearly seperated pieces of information for each solution (40 words in total):
    1) "Title": Repeat the titel of the idea
    2) "Summary": 3 sentence summary of the solution
    3) "Evaluation criteria 2 details": Identify possible sources of error. Look for risks, pointing out limits and gaps, and expressing constructive criticism. Examine its real-world implications from a more critical viewpoint. You want to make sure that your plan is as comprehensive as it can be. Every detail needs to be scrutinized and refined. (1 sentence only)
    4) "Evaluation criteria 2": Does it have many potential sources of errors? Answer can be 1,2 or 3 (1 = many sources of error, 2 = some sources of error, 3 = few sources of error) (be the most critical person ever. Only give max. 1 of the 3 solution a 3) (1 sentence)
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt2}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print("Critics Solutions:")    
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
critics_solutions[0] = completion.choices[0].message


#Summary

summary_solutions = {}

user_prompt3 = f"""

    Imagine you should give a summary to a high executive manager about a brainstorming session. 
    
    During the brainstorming session your team came up with the following ideas: {dreamer_solutions}. 
    For each idea you considered how feasible it is {realist_solutions}. 
    Last, you evaluated the amount of potential sources of errors: {critics_solutions} 
    
    Take a deep breath and work on this problem. 
    
    Please summarize in one paragraph for each solution, with the following content:
    
    “Name”: Solution name (3 words max)
    “Solved Problem”: 1 or 2 sentence summary of the problem that is being solved 
    “Solution”: Provide a summary of the solution and how it works (in 4-5 sentences). Be dilligent on this task and make sure yo use the length limit. 
    “Impact”: How does the solution solve the specific problem (in 2-3 sentences)
    “Technology”: Please also give a very detailed explanation of which technology is utilized and how it is utilized. Please be very diligent in this task, imagine you are explaining it to the CTO who has a background of engineering and has to implement the solution (in 4-5 sentences)
    “Sources”: Please provide the specific sources of inspiration that you used from the {summary_list} (1 sentence max)
    
    """
    

completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt},
{"role": "user", "content": user_prompt3}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

 
print("Summary of Solutions:")   
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
summary_solutions[0] = completion.choices[0].message



Dreamer Solutions:
### 1) Biodegradable Appliance Components

**Your solution summarized in 3 sentences:**

This solution involves creating home appliances with components made entirely of advanced biodegradable materials that can decompose safely and quickly once discarded. The materials would be engineered to maintain durability and functionality throughout the product's lifespan. At the end of the appliance's life, the components could be composted, reducing landfill waste and promoting a cycle of natural renewal.

**Your solution from the customer perspective (customer journey): how does it work for the customer):**

Customers would purchase these appliances just like any other, but with the knowledge that at the end of their useful life, they could be returned to the manufacturer or taken to a composting facility. This process would not only ease the guilt associated with disposing of large appliances but also engage customers in a larger ecosystem of sustainability. Additionally,

Realist Solutions:
### 1) Biodegradable Appliance Components

**Title:** Biodegradable Appliance Components

**Summary:** This solution proposes creating home appliances from advanced biodegradable materials that decompose safely post-use. These materials would ensure durability during the appliance's lifespan and allow for composting at life's end, reducing landfill waste.

**Evaluation criteria 1 detail:** Realizing this solution requires extensive R&D in materials science to develop biodegradable materials that are both durable and suitable for appliance use. Collaboration with biotechnologists, environmental scientists, and manufacturing experts is essential. Regulatory approval for new materials and establishing composting facilities capable of handling these appliances are also critical steps. Consumer education on proper disposal practices would be necessary to ensure environmental benefits are realized.

**Evaluation criteria 1:** 2

### 2) Self-Repairing Appliances

**Title:**

# Solution Evaluation

In [10]:
# Initial rating of the solutions

initial_rating = {}

system_prompt_2 = f"""You are a highly qualified critical rater of business opportunities. When evaluating the opportunities, you should take the perspective of a potential client of these solutions, active in the specific industry of {query}.
You are now tasked to evaluate a set of opportunities with the following problem in mind: {Problem} """


user_prompt4 = f""""

Task: Evaluate the provided solutions based on detailed criteria.

Input Text: {summary_solutions}

Instructions:

Extract Solutions: Begin by identifying all solutions within the input text. For each solution, note the following details:
    - Name
    - Solved Problem
    - Solution Description
    - Impact
    - Technology Used
    - Sources
    
Develop Evaluation Criteria: Create 10 specific criteria to assess the solutions. These criteria should enable a clear comparison between the solutions, focusing on aspects such as innovation, scalability, social impact, and cost-effectiveness.

Rate Each Solution: Apply the criteria to each solution, assigning ratings on a scale from 1 to 3 (1=Lowest, 3=Highest). Be stringent with the ratings:
    1: Does not meet expectations
    2: Meets expectations adequately
    3: Exceeds expectations significantly

A rating of 3 should be reserved for exceptional performance in the given criterion.

Then, also output the total score of a solution, out of the maximum of 30 points.

Repeat for All Solutions: Follow the steps above for each solution presented in the input text.

Objective: This structured analysis will facilitate a comprehensive comparison of the solutions, highlighting their strengths and areas for improvement based on the defined criteria.

Create a table which gives an overview of the solutions and their ratings in the different criteria, giving a number to each solution and only mentionning the number in the horizontal axis of the table. The solution name should not be included in the table.

In your output, please ensure that each solution is separated by "\n---\n".
"""


completion = openai.chat.completions.create(
model="gpt-4-0125-preview",
messages=[
{"role": "system", "content": system_prompt_2},
{"role": "user", "content": user_prompt4}
],
temperature=0.7,
max_tokens=2000,
top_p=0.95,
frequency_penalty=0,
presence_penalty=0,
stop=None
)

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability
initial_rating[0] = completion.choices[0].message



### Extracted Solutions

1. **Biodegradable Appliance Components**
   - **Name**: Eco-Decompose Appliances
   - **Solved Problem**: Environmental impact of appliance disposal
   - **Solution Description**: Manufacturing home appliances with biodegradable materials for safe composting post-use.
   - **Impact**: Reduces landfill waste and supports natural composting processes.
   - **Technology Used**: Advanced biodegradable materials science.
   - **Sources**: Cyclic Materials, e2ip technologies.

2. **Self-Repairing Appliances**
   - **Name**: Self-Heal Tech
   - **Solved Problem**: Appliance damage leading to waste
   - **Solution Description**: Appliances repair minor damages using nanotechnology and self-healing materials.
   - **Impact**: Extends appliance lifespan and reduces waste.
   - **Technology Used**: Nanotechnology and self-healing materials.
   - **Sources**: e2ip technologies.

3. **Modular Upgradeable Appliances**
   - **Name**: Modular Eco-Units
   - **Solved Problem**

In [11]:
# Pick the best three 

user_prompt5 = f"""
Extract and rank the solutions based on their overall ratings from the provided data: {initial_rating}.

Identify the top three solutions with the highest overall ratings out of 30 and sort them in descending order of their ratings.

For each of these top solutions, provide detailed information based on the content from: {summary_solutions}. Ensure comprehensive coverage of the points listed below:

- "Name": The solution's name, limited to a maximum of 5 words.
- "Solved Problem": A concise three-sentence summary of the problem addressed by the solution (minimum 50 words).
- "Solution": A brief summary of the solution, including its mechanism, in 4 to 5 sentences.
- "Impact": Explanation of how the solution addresses the specific problem, in 2 to 3 sentences.
- "Technology": Detailed description of the technologies used, including their application, aimed at a CTO with an engineering background for implementation purposes, in 4-5 sentences.
- "Sources": Mention any specific inspiration sources from {summary_list}, limited to 1 sentence.

Additionally, highlight the rating criteria where each solution excels.

Please separate each solution's details with "---".

"""

# Call the GPT model to generate the completion
completion = openai.chat.completions.create(
    model="gpt-4-0125-preview",
    messages=[
    {"role": "system", "content": system_prompt_2},
    {"role": "user", "content": user_prompt5}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

# Correctly access the generated text from the completion response
generated_text = completion.choices[0].message.content

# Split the generated text into individual solutions based on your specified separator ("---")
solutions_list = generated_text.split("---")

# Initialize an empty dictionary to hold your final solutions
final_solutions = {}

# Populate the dictionary with distinct solutions
for index, solution_text in enumerate(solutions_list, start=1):
    if solution_text.strip():  # Ensure the string is not just whitespace
        final_solutions[index] = solution_text.strip()

# Ensure only the top 3 solutions are kept if more were accidentally generated
final_solutions = dict(list(final_solutions.items())[:3])

# 'final_solutions' now contains up to three distinct items as requested

print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the messages for readability


### AI-Optimized Appliance Networks

**Name**: Smart Eco-Network  
**Solved Problem**: This solution addresses the inefficiency of appliance energy use by leveraging AI to optimize operations based on real-time data, aligning with consumer demand for energy-efficient and environmentally friendly home solutions. By adjusting appliance energy consumption patterns, it significantly reduces the environmental footprint of household appliance use.  
**Solution**: Through an AI-driven network, home appliances are interconnected and optimized to minimize environmental impact. The system dynamically adjusts operation times, energy sources, and efficiency settings in response to utility rates and renewable energy availability, promoting sustainable living.  
**Impact**: This innovation facilitates a significant reduction in energy consumption and environmental impact by managing appliance use more effectively, making sustainable living more accessible to consumers.  
**Technology**: The solution

# Integrate further Mergeflow Data

In [12]:
# Integrate new mergeflow data for the solutions 

# Now, based on soultions found, we need more info from Mergeflow


boolean_queries = {}

for index, solution_name in final_solutions.items():
    user_prompt6 = f"""
    
    Create a boolean query based on the Name of the solution from {solution_name}.
    
    Please use the following as an example and follow the format: 
    
    Example: for Factory Blockchain Protocol, this would be:

    "factory blockchain protocol" OR "blockchain for manufacturing" OR "blockchain in production" OR "blockchain in factories" OR "smart factory blockchain" OR "industrial blockchain" OR "supply chain blockchain" OR "blockchain logistics" OR "blockchain traceability"
    

    """
    
    completion = openai.chat.completions.create(
        model="gpt-4-0125-preview",
        messages=[
        {"role": "system", "content": system_prompt_2},
        {"role": "user", "content": user_prompt6}
        ],
        temperature=0.7,
        max_tokens=2000,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
        )

    # Extracting the content of the message from the first choice in the choices list
    query_content = completion.choices[0].message.content
    
    # Save the generated boolean query for the current solution
    boolean_queries[index] = query_content

    # Print the generated boolean query for the current solution
    print(query_content)
    print("\n----\n")
    

    

"smart eco-network" OR "AI-optimized appliance network" OR "intelligent home energy management" OR "IoT home appliances sustainability" OR "AI-driven energy efficiency" OR "smart home energy optimization" OR "environmentally friendly appliance network" OR "renewable energy appliance integration" OR "sustainable living AI technology" OR "energy consumption reduction AI"

----

"modular eco-units" OR "upgradeable appliances" OR "sustainable appliance solutions" OR "modular appliance technology" OR "eco-friendly appliance upgrades" OR "appliance lifespan extension" OR "electronic waste reduction" OR "modular home appliances" OR "technology upgrade subscription for appliances" OR "sustainable home technology" OR "environmentally friendly appliances"

----

"eco-decompose appliances" OR "biodegradable home appliances" OR "compostable appliance components" OR "sustainable appliance disposal" OR "environment-friendly home appliances" OR "biodegradable materials in appliances" OR "home applian

In [13]:
dataset = '&sp=3814'  # access dataset from mergeflow - in this case industry news
rows = '3'  # first 3 docs only

# Iterate through each boolean query
for solution, query_text in boolean_queries.items():
    # Correctly encode the query
    encoded_query = query_text.replace(" OR ", '%7C').replace('" ', '%22').replace(' "', '%22').replace(' ', '+').replace('"', '%22')

    # Construct the API request URL
    url = f'https://mergeflow.net/api/v1/{username}/search/getContent?q={encoded_query}{dataset}&rows={rows}'

    mergeflow_url = f'https://mergeflow.net/Profiles/{username}/search?date=default3&q={encoded_query}{dataset}&rows={rows}'

    
    # Define the headers to be sent with the request
    headers = {
        'MergeflowNet-Api-Auth-Key': mergeflow_api_key
    }

    # Send the POST request
    response = requests.post(url, headers=headers)

    mergeflow_docs = []

    # Check the response status code and content
    if response.status_code == 200:
        mergeflow_response_json = response.json()
        
        # Extract and store documents information
        for document in mergeflow_response_json['Documents']:
            current_doc = {
                'title': document['Title'],
                'date': document['Date'],
                'url': document['Url'],
                'content': document['Content']
            }
            
            mergeflow_docs.append(current_doc)
            


    # Process or print your results here
    # For example, print titles of documents for this query
    print(f"Results for Solution {solution}:")
    print("\n-\n") 
    print(f"Mergeflow URL:")
    print(mergeflow_url)
    print("\n-\n") 

    for doc in mergeflow_docs:
        print(doc['title'])
        print(doc['url'])
    print("\n----\n")
    

    

Results for Solution 1:

-

Mergeflow URL:
https://mergeflow.net/Profiles/leonard/search?date=default3&q=%22smart+eco-network%22%7C%22AI-optimized+appliance+network%22%7C%22intelligent+home+energy+management%22%7C%22IoT+home+appliances+sustainability%22%7C%22AI-driven+energy+efficiency%22%7C%22smart+home+energy+optimization%22%7C%22environmentally+friendly+appliance+network%22%7C%22renewable+energy+appliance+integration%22%7C%22sustainable+living+AI+technology%22%7C%22energy+consumption+reduction+AI%22&sp=3814&rows=3

-

CORRECTING and REPLACING Lumin’s Smart Electrical Panel Selected by Sunnova to Supercharge its Solar and Storage Offering | Business Wire
http://www.businesswire.com/news/home/20240130077209/en/CORRECTING-and-REPLACING-Lumin%E2%80%99s-Smart-Electrical-Panel-Selected-by-Sunnova-to-Supercharge-its-Solar-and-Storage-Offering/
Schneider Electric’s Wiser Home Energy Management App Named 2023 CES Innovation Award Honoree | Business Wire
http://www.businesswire.com/news/home/